In [113]:
import os
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [114]:

# Define the base folder
base_folder = 'Multimodal_images'

# Use glob to find all jpg files recursively
jpg_files = glob.glob(os.path.join(base_folder, '**', '*.jpg'), recursive=True)

# Print the list of jpg file paths
print(jpg_files[0])


Multimodal_images\cyanosis\Image_1.jpg


In [115]:
len(jpg_files)

532

In [116]:
# Function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    labels = []
    for path in image_paths:
        image = load_img(path, target_size=target_size)   #module come from tensorflow.keras.preprocessing.image
        image = img_to_array(image)  # (height x width x channels) = (224,224,3)
        images.append(image)
        label = os.path.basename(os.path.dirname(path))  # Assuming folder name is the label
        labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [117]:
jpg_files[0]

'Multimodal_images\\cyanosis\\Image_1.jpg'

In [118]:
# Load the images and their labels
image_paths =jpg_files
images, labels = load_and_preprocess_images(image_paths)


In [119]:
labels[:4]

array(['cyanosis', 'cyanosis', 'cyanosis', 'cyanosis'], dtype='<U15')

In [120]:
print(len(images))
print(len(images[0]))
print(len(images[0][0]))
print(len(images[0][0][0]))

532
224
224
3


In [121]:
images[0]

array([[[  9.,  13.,  12.],
        [  6.,  10.,   9.],
        [  7.,  11.,  10.],
        ...,
        [177., 226., 205.],
        [182., 227., 207.],
        [181., 230., 209.]],

       [[  6.,  10.,   9.],
        [  4.,   8.,   7.],
        [  6.,  10.,   9.],
        ...,
        [177., 226., 205.],
        [182., 227., 207.],
        [181., 230., 209.]],

       [[  7.,  11.,  10.],
        [  5.,   9.,   8.],
        [  5.,   9.,   8.],
        ...,
        [179., 228., 207.],
        [180., 225., 205.],
        [179., 228., 207.]],

       ...,

       [[103., 168., 206.],
        [ 92., 156., 192.],
        [ 91., 155., 193.],
        ...,
        [206., 171., 149.],
        [201., 174., 147.],
        [200., 173., 144.]],

       [[ 66., 135., 168.],
        [ 55., 121., 155.],
        [ 33.,  93., 129.],
        ...,
        [203., 171., 148.],
        [206., 176., 152.],
        [201., 175., 150.]],

       [[ 59., 125., 159.],
        [ 48., 111., 146.],
        [ 41.,  

In [122]:

images.shape

(532, 224, 224, 3)

In [123]:
# Preprocess images for ResNet50
images = tf.keras.applications.resnet50.preprocess_input(images) 
#For ResNet50, this involves converting the pixel values from a range of [0, 255] to a range suitable for the model.
# "(1)Convert RGB to BGR and (2)convert center at 0 "

In [124]:
images[0]

array([[[ -91.939    , -103.779    , -114.68     ],
        [ -94.939    , -106.779    , -117.68     ],
        [ -93.939    , -105.779    , -116.68     ],
        ...,
        [ 101.061    ,  109.221    ,   53.32     ],
        [ 103.061    ,  110.221    ,   58.32     ],
        [ 105.061    ,  113.221    ,   57.32     ]],

       [[ -94.939    , -106.779    , -117.68     ],
        [ -96.939    , -108.779    , -119.68     ],
        [ -94.939    , -106.779    , -117.68     ],
        ...,
        [ 101.061    ,  109.221    ,   53.32     ],
        [ 103.061    ,  110.221    ,   58.32     ],
        [ 105.061    ,  113.221    ,   57.32     ]],

       [[ -93.939    , -105.779    , -116.68     ],
        [ -95.939    , -107.779    , -118.68     ],
        [ -95.939    , -107.779    , -118.68     ],
        ...,
        [ 103.061    ,  111.221    ,   55.32     ],
        [ 101.061    ,  108.221    ,   56.32     ],
        [ 103.061    ,  111.221    ,   55.32     ]],

       ...,

      

In [125]:
# images[0]

In [126]:
images.shape

(532, 224, 224, 3)

In [127]:
# Encode labels
label_encoder = LabelEncoder()
print(label_encoder)
#For example, if labels are ['cat', 'dog', 'mouse'], it might convert them to [0, 1, 2]
labels_encoded = label_encoder.fit_transform(labels) 
print(labels_encoded)
# len(labels_encoded)
#Convert Numerical Labels to One-Hot Encoding:
labels_categorical = to_categorical(labels_encoded)
labels_categorical

LabelEncoder()
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 13 13 13 13 13 13 13

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [128]:
len(labels_encoded)

532

In [129]:
labels_categorical.shape

(532, 18)

In [130]:
print(len(labels_categorical))
print(len(labels_categorical[0]))

532
18


In [131]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)


In [132]:
# Load ResNet50 model without the top fully connected layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [133]:

len(label_encoder.classes_)

18

In [134]:
# Add custom layers on top of the base model
model = Sequential([
    base_model,
    GlobalMaxPooling2D(),
    Dense(1024, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [135]:
# Freeze the base model layers to avoid retraining them. 
# Means weight unchange(take weight from orginal renest50) in new sequential model
base_model.trainable = False

In [136]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [137]:
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
14/14 [==============================] - 39s 3s/step - loss: 21.9223 - accuracy: 0.2871 - val_loss: 11.5860 - val_accuracy: 0.3551
Epoch 2/10
14/14 [==============================] - 33s 2s/step - loss: 4.7754 - accuracy: 0.5647 - val_loss: 4.2136 - val_accuracy: 0.4766
Epoch 3/10
14/14 [==============================] - 31s 2s/step - loss: 1.3118 - accuracy: 0.7976 - val_loss: 2.9762 - val_accuracy: 0.5421
Epoch 4/10
14/14 [==============================] - 32s 2s/step - loss: 0.7281 - accuracy: 0.8188 - val_loss: 2.7755 - val_accuracy: 0.5047
Epoch 5/10
14/14 [==============================] - 30s 2s/step - loss: 0.3141 - accuracy: 0.9129 - val_loss: 1.9217 - val_accuracy: 0.6449
Epoch 6/10
14/14 [==============================] - 30s 2s/step - loss: 0.0733 - accuracy: 0.9718 - val_loss: 2.4681 - val_accuracy: 0.5701
Epoch 7/10
14/14 [==============================] - 30s 2s/step - loss: 0.0305 - accuracy: 0.9929 - val_loss: 2.3348 - val_accuracy: 0.6262
Epoch 8/10
14/14 [

In [138]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

4/4 [==============================] - 7s 2s/step - loss: 1.7398 - accuracy: 0.6729
Test accuracy: 67.29%


In [139]:
# Save the model
#model.save('custom_resnet50_model.h5')

In [140]:
image, labels = load_and_preprocess_images(['images1.jpg'])
    
# Predict probabilities
preds = model.predict(image)
    
# Get the predicted class label
pred_class = np.argmax(preds)
    
# Map predicted class index to class label
pred_label = label_encoder.inverse_transform([pred_class])[0]

print(pred_label)

1/1 [==============================] - 1s 906ms/step
hand lump
